In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
import random

num_of_centroids=4096
num_train_samples=500
num_test_samples=125
K = 11
all_image_idx=random.sample(range(0,1000),num_train_samples+num_test_samples)

In [7]:
from PIL import Image
from skimage.filters import unsharp_mask

def remove_noise (image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    median = cv2.medianBlur(gray, 3)
    result_2 = unsharp_mask(median, radius=10, amount=4)*255
    result_2 = np.uint8(result_2)
    sharpen = cv2.Canny(result_2, 100,250)
    return sharpen

def resize(image, width=200, hsize=None):
    # Function to resize an image

    base_width = width

    # Calculate the width percentage to maintain the aspect ratio
    width_percent = (base_width / float(image.size[0]))
    # If hsize is not provided, calculate the height based on the width percentage
    hsize = int(
        (float(image.size[1]) * float(width_percent))) if hsize == None else hsize
    # Resize the image using the calculated width and height
    new_size = (base_width, hsize)
    image = image.resize(new_size, Image.LANCZOS)
    return image

def calculate_brightness(gray, index=None):
    # Function to calculate the brightness of an image

    # Calculate the mean brightness value of the grayscale image
    mean_value = cv2.mean(gray)[0]

    # If an index is provided, display the grayscale image and print the mean value
    if index is not None:
        show_images([gray], [str(index)])
        # print("the image "+str(index)+" has mean value : ", mean_value)

    # Return the mean brightness value
    return mean_value

def get_binary_high_constract(img_RGB):
    # Function to get the binary image with high contrast

    # img_RGBA = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2RGBA)
    # img_gray = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2GRAY)
    # ret, img_binary = cv2.threshold(img_gray, 60, 255, cv2.THRESH_BINARY_INV)
    # img_HSV = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2HSV)
    # img_YCC = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2YCR_CB)
    # img_BGR = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2BGR)

    # Convert RGB image to YUV color space
    img_YUV = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2YUV)

    # Make float and divide by 255 to give BGRdash
    # bgrdash = img_BGR.astype(np.float)/255.

    # hls_img = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2HLS)
    # lab_img = cv.cvtColor(img_RGB, cv.COLOR_RGB2LAB)

    # img_HSV[:, :, 0] = cv2.equalizeHist(img_HSV[:, :, 0])
    # img_HSV[:, :, 1] = cv2.equalizeHist(img_HSV[:, :, 1])
    # img_HSV[:, :, 2] = cv2.equalizeHist(img_HSV[:, :, 2])

    # img_YCC[:, :, 0] = cv2.equalizeHist(img_YCC[:, :, 0])
    # img_YCC[:, :, 1] = cv2.equalizeHist(img_YCC[:, :, 1])
    # img_YCC[:, :, 2] = cv2.equalizeHist(img_YCC[:, :, 2])

    #     img_YUV[:, :, 0] = cv2.equalizeHist(img_YUV[:, :, 0])
    #     img_YUV[:, :, 1] = cv2.equalizeHist(img_YUV[:, :, 1])

    # Equalize the Y channel of the YUV image
    img_YUV[:, :, 2] = cv2.equalizeHist(img_YUV[:, :, 2])

    # hls_img[:, :, 0] = cv2.equalizeHist(hls_img[:, :, 0])
    # hls_img[:, :, 1] = cv2.equalizeHist(hls_img[:, :, 1])
    # hls_img[:, :, 2] = cv2.equalizeHist(hls_img[:, :, 2])

    # lab_img[:, :, 0] = cv2.equalizeHist(lab_img[:, :, 0])
    # lab_img[:, :, 1] = cv2.equalizeHist(lab_img[:, :, 1])
    # lab_img[:, :, 2] = cv2.equalizeHist(lab_img[:, :, 2])

    # Apply thresholding to obtain a binary image
    ret, img_binary1 = cv2.threshold(
        img_YUV[:, :, 2], 150, 255, cv2.THRESH_BINARY)

    # Create a structuring element for morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

    # Perform erosion and dilation on the binary image
    dilate_img = cv2.erode(img_binary1, kernel, iterations=1)
    erode_img = cv2.dilate(dilate_img, kernel, iterations=1)

    # Return the binary image with high contrast
    return erode_img

def get_binary_low_medium_constract(img_RGB):
    # Function to get the binary image with low to medium contrast

    gray = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2GRAY)
    img_HSV = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2HSV)

    # Thresholding the saturation channel of the HSV image
    ret, img_binary = cv2.threshold(
        img_HSV[:, :, 1], 50, 255, cv2.THRESH_BINARY)

    # Thresholding the value channel of the HSV image
    ret, img_binary2 = cv2.threshold(
        img_HSV[:, :, 2], 50, 255, cv2.THRESH_BINARY)

    # Combining the two binary images
    img_binary[img_binary2 == 0] = 0

    # Create a structuring element for morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

    # Apply erosion to the image
    erode_img = cv2.erode(img_binary, kernel, iterations=2)

    # Apply dilation to the eroded image
    dilate_img = cv2.dilate(erode_img, kernel, iterations=2)
    return dilate_img


def reduce_image(img, thickness):
    # Function to reduce the size of an image

    mask1 = img.copy()

    # Set the top thickness rows to 0 (black)
    mask1[:thickness, :] = 0

    # Set the bottom thickness rows to 0 (black)
    mask1[-1*thickness:, :] = 0

    # Set the left thickness columns to 0 (black)
    mask1[:, :thickness] = 0

    # Set the right thickness columns to 0 (black)
    mask1[:, -1*thickness:] = 0
    return mask1

def get_result(img, mask):
    # Function to get the result image with a given mask

    copy = img.copy()
    channels = img.shape[2]
    # Apply bitwise AND operation between each channel of the image and the mask
    for i in range(channels):
        copy[:, :, i] = np.bitwise_and(copy[:, :, i], mask)
    temp = mask.copy()
    # Convert 0 values in the mask to 255 (white) and 255 values to 0 (black)
    temp[mask == 0] = 255
    temp[mask == 255] = 0
    # Apply bitwise OR operation between each channel of the image and the modified mask
    for i in range(channels):
        copy[:, :, i] = np.bitwise_or(copy[:, :, i], temp)
    return copy

def remove_noise2 (image):
    
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize(image, 4 * 128, 4 * 64)

    # Convert the resized image to RGB and grayscale
    img_RGB = np.array(image)
    gray = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2GRAY)

    # Calculate the mean brightness value of the grayscale image
    mean_value = calculate_brightness(gray)
    dilate_img = None

    # Determine the appropriate image processing based on the mean brightness value
    if (mean_value >= 190):
        dilate_img = get_binary_high_constract(img_RGB)
    else:
        dilate_img = get_binary_low_medium_constract(img_RGB)

    # Convert the dilated image to binary (0 or 1) and create a mask
    dilate_img[dilate_img == 255] = 1
    mask1 = dilate_img
    mask1[mask1 != 1] = 0
    mask1[mask1 == 1] = 255

    # cut the border of the image, ---------------- it was originally mask1
    mask1 = reduce_image(img=gray, thickness=3)

    # Apply Gaussian blur and Canny edge detection to the mask
    blured = cv2.GaussianBlur(mask1, (5, 5), 0)
    edges = cv2.Canny(blured, threshold1=40, threshold2=100,
                      apertureSize=3, L2gradient=False)

    # Create a kernel for morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

    # Close the edges using morphological closing
    closed_edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    # Find contours in the closed edges
    contours, hierarchy = cv2.findContours(
        closed_edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    # Create a mask and find the largest contour
    mask = np.zeros_like(edges)
    max_area = 0
    second_max = 0
    biggest_contour = None
    second_biggest = None
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > max_area:
            max_area = area
            biggest_contour = cnt
    # If there is at least one contour, draw it on the mask
    if len(contours) > 0:
        mask = cv2.drawContours(
            mask, [biggest_contour], -1, (255, 255, 255), -1)

    # Generate the result by applying the mask to the original image
    result = get_result(img_RGB, mask)

    # Apply a 5x5 averaging kernel to the result
    kernel = np.ones((5, 5), np.float32)/25
    result = cv2.filter2D(result, -1, kernel)
    if len(contours) > 0:
        (x, y, w, h) = cv2.boundingRect(biggest_contour)
        result = result[y:y+h, x:x+w]
        mask = mask[y:y+h, x:x+w]
        # print(x, y, w, h)

    # Resize the result to the desired dimensions and convert to grayscale
    result = cv2.resize(result, (128, 64))
    result = cv2.cvtColor(result, cv2.COLOR_RGB2GRAY)
    #     show_images([ result, rotated_img])
    # Return the mask and preprocessed result image as a tuple
    # show all the generated images
    show_images([img_RGB, gray, dilate_img, mask1, blured, edges, closed_edges, mask, result])
    return result

def show_images(images, titles=None):
    n_ims = len(images)
    if titles is None:
        titles = ['(%d)' % i for i in range(1, n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image, title in zip(images, titles):
        a = fig.add_subplot(1, n_ims, n)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()

In [8]:
all_des=np.empty((1,128))

sift = cv2.SIFT_create()

num_of_desc=[]

for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/IBM Plex Sans Arabic/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 1 image"+str(i))

num_of_desc+=[all_des.shape[0]]
print("FINISHED READING FIRST SET OF IMAGES")

for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/Lemonada/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 2 image"+str(i))

num_of_desc+=[all_des.shape[0]]
print("FINISHED READING SECOND SET OF IMAGES")

for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/Marhey/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 3 image"+str(i))

num_of_desc+=[all_des.shape[0]]
print("FINISHED READING THIRD SET OF IMAGES")


for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/Scheherazade New/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 4 image"+str(i))

num_of_desc+=[all_des.shape[0]]
print("FINISHED READING FOURTH SET OF IMAGES")

desc_labels=np.zeros(all_des.shape[0])
desc_labels[num_of_desc[0]:num_of_desc[1]]=1
desc_labels[num_of_desc[1]:num_of_desc[2]]=2
desc_labels[num_of_desc[2]:num_of_desc[3]]=3

processing type 1 image0
processing type 1 image1
processing type 1 image2
processing type 1 image3
processing type 1 image4
processing type 1 image5
processing type 1 image6
processing type 1 image7
processing type 1 image8
processing type 1 image9
processing type 1 image10
processing type 1 image11
processing type 1 image12
processing type 1 image13
processing type 1 image14
processing type 1 image15
processing type 1 image16
processing type 1 image17
processing type 1 image18
processing type 1 image19
processing type 1 image20
processing type 1 image21
processing type 1 image22
processing type 1 image23
processing type 1 image24
processing type 1 image25
processing type 1 image26
processing type 1 image27
processing type 1 image28
processing type 1 image29
processing type 1 image30
processing type 1 image31
processing type 1 image32
processing type 1 image33
processing type 1 image34
processing type 1 image35
processing type 1 image36
processing type 1 image37
processing type 1 imag

In [ ]:
# different options for kmeans
# - mini batches
# - limiting the number of descriptors
kmeans=MiniBatchKMeans(n_clusters=num_of_centroids,batch_size=num_of_centroids*1024,max_iter=20).fit(X=all_des)

D:\Third_Year\Neural\Labs\env\neural\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "D:\Third_Year\Neural\Labs\env\neural\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 1026, in __init__
    self._execute_c

In [ ]:
# for each centroid, calculate how it is near to each label
centroids_labels=np.zeros((num_of_centroids,4))
for i in range(num_of_centroids):
    centroids_labels[i][0]=(np.sum(desc_labels[kmeans.labels_==i]==0))  ## kmeans.labels_ -> array to map each descriptor to a centroid
    centroids_labels[i][1]=(np.sum(desc_labels[kmeans.labels_==i]==1))
    centroids_labels[i][2]=(np.sum(desc_labels[kmeans.labels_==i]==2))
    centroids_labels[i][3]=(np.sum(desc_labels[kmeans.labels_==i]==3))
    centroids_labels[i]/=np.sum(centroids_labels[i])
print(centroids_labels[0][0], centroids_labels[0][1], centroids_labels[0][2], centroids_labels[0][3])

In [ ]:
def predict(path,centroids,centroids_labels):
    image=cv2.imread(path)
    removed_noise = remove_noise(image)
    _ , descriptors= sift.detectAndCompute(removed_noise,None)

    predections=[0.0,0.0,0.0,0.0]
    if descriptors is not None:
        for des in descriptors:
            idx=kmeans.predict([des])
            dist=np.linalg.norm(des-centroids[idx])
            if dist == 0:
                dist = 0.0000001
            predections+=(centroids_labels[idx]/dist)
        return np.argmax(predections)
    else:
        return -1

num_right0=0
num_right1=0
num_right2=0
num_right3=0

predictions=[]

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])
    path="../../fonts-dataset/IBM Plex Sans Arabic/"+rand_idx+".jpeg"

    predicted=predict(path,kmeans.cluster_centers_,centroids_labels)
    print("image",(rand_idx), "type0: ",predicted)
    num_right0+=predicted==0
    predictions.append(predicted)

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])
    path="../../fonts-dataset/Lemonada/"+rand_idx+".jpeg"

    predicted=predict(path,kmeans.cluster_centers_,centroids_labels)
    print("image",(rand_idx), "type1: ",predicted)
    num_right1+=predicted==1
    predictions.append(predicted)

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])

    path="../../fonts-dataset/Marhey/"+rand_idx+".jpeg"

    predicted=predict(path,kmeans.cluster_centers_,centroids_labels)
    print("image",(rand_idx), "type2: ",predicted)
    num_right2+=predicted==2
    predictions.append(predicted)

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])

    path="../../fonts-dataset/Scheherazade New/"+rand_idx+".jpeg"

    predicted=predict(path,kmeans.cluster_centers_,centroids_labels)
    print("image",(rand_idx), "type3: ",predicted)
    num_right3+=predicted==3
    predictions.append(predicted)

In [ ]:
print(num_right0)
print(num_right1)
print(num_right2)
print(num_right3)

print((num_right0+num_right1+num_right2+num_right3)/(4*num_test_samples))

In [ ]:
from sklearn.neighbors import NearestNeighbors

def predict_knn(path,centroids_labels,knn):
    image=cv2.imread(path)
    removed_noise = remove_noise(image)
    _ , descriptors= sift.detectAndCompute(removed_noise,None)

    predections=[0.0,0.0,0.0,0.0]
    if descriptors is not None:
        for des in descriptors:
            neigh=knn.kneighbors([des])
            for i in range(K):
                predections+=(centroids_labels[neigh[1][0][i]]/neigh[0][0][i])
        print(predections)
        return np.argmax(predections)
    else:
        return -1

knn = NearestNeighbors(n_neighbors=K)
classes=np.arange(0, num_of_centroids, 1, dtype=int)
knn.fit(kmeans.cluster_centers_,classes)

predictions=[]

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])
    path="../../fonts-dataset/IBM Plex Sans Arabic/"+rand_idx+".jpeg"

    predicted=predict_knn(path,centroids_labels,knn)
    print("image",(rand_idx), "type0: ",predicted)
    num_right0+=predicted==0
    predictions+=predicted

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])
    path="../../fonts-dataset/Lemonada/"+rand_idx+".jpeg"

    predicted=predict_knn(path,centroids_labels,knn)
    print("image",(rand_idx), "type1: ",predicted)
    num_right1+=predicted==1
    predictions+=predicted

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])

    path="../../fonts-dataset/Marhey/"+rand_idx+".jpeg"

    predicted=predict_knn(path,centroids_labels,knn)
    print("image",(rand_idx), "type2: ",predicted)
    num_right2+=predicted==2
    predictions+=predicted

for i in range (num_test_samples):
    rand_idx=str(all_image_idx[num_train_samples+i])

    path="../../fonts-dataset/Scheherazade New/"+rand_idx+".jpeg"

    predicted=predict_knn(path,centroids_labels,knn)
    print("image",(rand_idx), "type3: ",predicted)
    num_right3+=predicted==3
    predictions+=predicted

In [ ]:
print(num_right0)
print(num_right1)
print(num_right2)
print(num_right3)

print((num_right0+num_right1+num_right2+num_right3)/(4*num_test_samples))

In [ ]:
# SVM on ther original des

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

all_des=[]
labels=[]
sift = cv2.SIFT_create(500)

for i in range (10):
    image = cv2.imread("../../fonts-dataset/Lemonada/"+str(i)+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    
    if descriptors is not None:
        for des in descriptors:
            all_des.append(des)
        # append number of labels to the labels array equal to the number of descriptors
        for j in range (descriptors.shape[0]):
            labels=np.append(labels,0)        
    


    image2 = cv2.imread("../../fonts-dataset/Marhey/"+str(i)+".jpeg")
    removed_noise2 = remove_noise(image2)
    kp , descriptors= sift.detectAndCompute(removed_noise2,None)

    if descriptors is not None:
        for des in descriptors:
            all_des.append(des)  
        # append number of labels to the labels array equal to the number of descriptors
        for j in range (descriptors.shape[0]):
            labels=np.append(labels,1)

    image3 = cv2.imread("../../fonts-dataset/IBM Plex Sans Arabic/"+str(i)+".jpeg")
    removed_noise3 = remove_noise(image3)
    kp , descriptors= sift.detectAndCompute(removed_noise3,None)

    if descriptors is not None:
        for des in descriptors:
            all_des.append(des)
        # append number of labels to the labels array equal to the number of descriptors
        for j in range (descriptors.shape[0]):
            labels=np.append(labels,2)

    image4 = cv2.imread("../../fonts-dataset/Scheherazade New/"+str(i)+".jpeg")
    removed_noise4 = remove_noise(image4)
    kp , descriptors= sift.detectAndCompute(removed_noise4,None)

    if descriptors is not None:
        for des in descriptors:
            all_des.append(des)
        # append number of labels to the labels array equal to the number of descriptors
        for j in range (descriptors.shape[0]):
            labels=np.append(labels,3)
    
    print(i)

all_des = np.array(all_des)
print(all_des.shape)

X_train, X_test, y_train, y_test = train_test_split(all_des, labels, test_size=0.2, random_state=42)
svm = SVC(kernel='poly', C=0.1, random_state=0, coef0=1, degree=4, gamma=10.0,class_weight= None)
print("fares1")
svm.fit(X_train, y_train)
print("fares1")
y_pred = svm.predict(X_test)
print(accuracy_score(y_test, y_pred)*100)

In [1]:
# Fixing individual predictions
def remove_noise22 (image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    X = cv2.bilateralFilter(gray, 15, sigmaColor=10, sigmaSpace=10)
    median = cv2.medianBlur(X, 5)
    result_2 = unsharp_mask(median, radius=10, amount=4)*255
    result_2 = np.uint8(result_2)
    sharpen = cv2.Canny(result_2, 100,250)
    
    # Create a kernel for morphological operations
    # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    # 
    # # Close the edges using morphological closing
    # closed_edges = cv2.morphologyEx(sharpen, cv2.MORPH_CLOSE, kernel)
    return sharpen

def predict22(path,centroids,centroids_labels):
    image=cv2.imread(path)
    removed_noise = remove_noise22(image)
    _ , descriptors= sift.detectAndCompute(removed_noise,None)

    predections=[0.0,0.0,0.0,0.0]
    if descriptors is not None:
        for des in descriptors:
            idx=kmeans.predict([des])
            dist=np.linalg.norm(des-centroids[idx])
            if dist == 0:
                dist = 0.0000001
            predections+=(centroids_labels[idx]/dist)
        print(predections)
        return np.argmax(predections)
    else:
        return -1
path="../../fonts-dataset/Scheherazade New/840.jpeg"

prediction=predict22(path,kmeans.cluster_centers_,centroids_labels)
print("image",(599), "type3: ",prediction)


NameError: name 'kmeans' is not defined

In [ ]:
print("image",(599), "type3: ",prediction)

In [198]:
image = cv2.imread("../../fonts-dataset/Scheherazade New/599.jpeg")
removed_noise = remove_noise22(image)
# save the image
cv2.imwrite("1.jpeg", removed_noise)

grayy = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
sharpen = cv2.Canny(grayy, 100,250)
cv2.imwrite("2.jpeg", sharpen)

result_2 = unsharp_mask(grayy, radius=10, amount=4)*255
result_2 = np.uint8(result_2)
#X = cv2.bilateralFilter(result_2, 15, sigmaColor=10, sigmaSpace=10)

# kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
# 
# # Close the edges using morphological closing
# closed_edges = cv2.morphologyEx(grayy, cv2.MORPH_DILATE, kernel)

# median = cv2.medianBlur(X, 5)
cv2.imwrite("3.jpeg", result_2)

True